In [3]:
from dataConnection import DataConnector
from glob import glob
files = glob("../data/dataset/supply_chain/*.xls")

In [4]:
import warnings
warnings.filterwarnings(action='ignore')

from datetime import datetime

from mlxtend.frequent_patterns import association_rules, apriori, fpgrowth
from mlxtend.preprocessing import TransactionEncoder

import sklearn
from sklearn.decomposition import TruncatedSVD

import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus']= False 

import platform
if platform.system() == 'Darwin': 
    plt.style.use('seaborn-darkgrid') 
    rc('font', family = 'AppleGothic') 

In [5]:
connection=DataConnector(files)
df=connection.df_preprocess()

Connecting
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (6693331) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size

In [6]:
df

,day,customer,product,quantity,aggregate,ym
0,2020-02-01,군산봉정요양병원**,1042594,3.0,3150.0,202002
1,2020-02-01,군산봉정요양병원**,1028277,2.0,3900.0,202002
2,2020-02-01,군산봉정요양병원**,1004327,110.0,10890.0,202002
3,2020-02-01,군산봉정요양병원**,1049419,1.0,14839.0,202002
4,2020-02-01,군산봉정요양병원**,1038658,1.0,3355.0,202002
...,...,...,...,...,...,...
33536,2020-11-30,xx늘푸른요양병원,1028336,2.0,5880.0,202011
33537,2020-11-30,xx늘푸른요양병원,1050467,9.0,15840.0,202011
33538,2020-11-30,xx늘푸른요양병원,1087500,4.0,10160.0,202011
33540,2020-11-30,xx늘푸른요양병원,1087488,1.0,14300.0,202011


# Dataset 1.
### Condition
#### - Latest 3months
#### - Over 75% of product quantile

In [7]:
gunsan=df[df['customer']=='군산봉정요양병원**']
gunsan=gunsan[['customer','product','ym']]
gunsan

,customer,product,ym
0,군산봉정요양병원**,1042594,202002
1,군산봉정요양병원**,1028277,202002
2,군산봉정요양병원**,1004327,202002
3,군산봉정요양병원**,1049419,202002
4,군산봉정요양병원**,1038658,202002
...,...,...,...
32188,군산봉정요양병원**,1049107,202011
32189,군산봉정요양병원**,1000232,202011
32190,군산봉정요양병원**,1125630,202011
32191,군산봉정요양병원**,1120305,202011


In [8]:
grouped_ym=gunsan.groupby('ym').agg(list)
# grouped_ym
len(grouped_ym['product'][-1])
# grouped_ym.to_frame().reset_index()

956

In [23]:
lst_length=[]
for i in range(len(grouped_ym['product'])):
    length=len(set(grouped_ym['product'][i]))
    lst_length.append(length)
print(max(lst_length), min(lst_length))


364 285


In [24]:
months=str(int(max(df["ym"]))-int(2))
gunsan_6months = gunsan[gunsan["ym"] >= months]
# tgunsan_6months.groupby('ym').agg(list)


In [25]:
quantile_75=(gunsan_6months['product'].value_counts()).quantile(q=0.75)
quantile_75

5.0

In [26]:
gunsan_proprecessed=gunsan_6months[gunsan_6months['product'].map(gunsan_6months['product'].value_counts())>quantile_75]
gunsan_proprecessed

,customer,product,ym
105,군산봉정요양병원**,1027181,202110
106,군산봉정요양병원**,1121019,202110
107,군산봉정요양병원**,1042594,202110
108,군산봉정요양병원**,1109397,202110
109,군산봉정요양병원**,1004291,202110
...,...,...,...
46227,군산봉정요양병원**,1121714,202112
46229,군산봉정요양병원**,1047170,202112
46230,군산봉정요양병원**,1086643,202112
46231,군산봉정요양병원**,1121115,202112


In [29]:
gunsan_product=gunsan_proprecessed.groupby('ym').agg(list)
gunsan_product

,customer,product
ym,,
202110,"[군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원*...","[1027181, 1121019, 1042594, 1109397, 1004291, ..."
202111,"[군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원*...","[1045177, 1040955, 1028140, 1109397, 1042594, ..."
202112,"[군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원**, 군산봉정요양병원*...","[1004291, 1121019, 1042594, 1109397, 1004291, ..."


# Temporary check svd

In [51]:
arr=np.array(dataset)
# print(arr)
# corr_mat = np.corrcoef(arr)
# corr_mat.shape

In [122]:
# temp['product'][0]

In [30]:
lst_length=[]
for i in range(len(gunsan_product['product'])):
    length=len(set(gunsan_product['product'][i]))
    lst_length.append(length)
print(max(lst_length), min(lst_length))


116 112


In [36]:
dataset=[]
for c,p in gunsan_product.iterrows():
    dataset.append(p['product'])


In [32]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
gunsan_dataset = pd.DataFrame(te_ary, columns=te.columns_)
gunsan_dataset

,1001574,1002205,1002213,1002291,1003327,1004291,1005406,1019595,1019909,1021177,...,1140506,1140516,1141891,1141894,1142526,1142536,1142549,1142551,1142592,1142598
0,True,True,True,True,True,True,True,True,True,True,...,False,True,True,True,True,True,True,True,True,True
1,True,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,False,True,True,True,True,True,True,True,True


In [33]:
%%time
apriori(gunsan_dataset, min_support=1.0, use_colnames=True)

KeyboardInterrupt: 

In [34]:
%%time
fpgrowth(gunsan_dataset, min_support=1.0, use_colnames=True)

KeyboardInterrupt: 

# Dataset 2.
### Condition
#### - Latest 6months
#### - Over 75% of product quantile

In [23]:
quantile_75=(gunsan_6months['product'].value_counts()).quantile(q=0.75)
quantile_75

7.0

In [26]:
# cut off the data by the frequency that is over 75% quantile
# gunsan_proprecessed=gunsan_6months[gunsan_6months['product'].map(gunsan_6months['product'].value_counts())>quantile_75].drop_duplicates(subset=['product'])
gunsan_proprecessed=gunsan_6months[gunsan_6months['product'].map(gunsan_6months['product'].value_counts())>quantile_75]
gunsan_proprecessed


,customer,product
105,군산봉정요양병원**,1027181
106,군산봉정요양병원**,1121019
107,군산봉정요양병원**,1042594
108,군산봉정요양병원**,1109397
109,군산봉정요양병원**,1004291
...,...,...
42498,군산봉정요양병원**,1042594
42499,군산봉정요양병원**,1004291
42500,군산봉정요양병원**,1028143
42501,군산봉정요양병원**,1121714


In [27]:
gunsan_proprecessed['product'].value_counts()

1109397    151
1004291    151
1121019    148
1042594    144
1028156    115
          ... 
1142859      8
1079351      8
1038443      8
1025335      8
1041843      8
Name: product, Length: 159, dtype: int64

In [28]:
onehot_set2=gunsan_proprecessed.pivot_table(index='customer', columns='product', aggfunc=len, fill_value=0)
onehot_set2=onehot_set2>0
onehot_set2

product,1000112,1000284,1000419,1001402,1001574,1002205,1002213,1003327,1004291,1005406,...,1141891,1141894,1142525,1142526,1142536,1142549,1142551,1142592,1142598,1142859
customer,,,,,,,,,,,,,,,,,,,,,
군산봉정요양병원**,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [32]:
%%time
frequent_itemsets_set2 = apriori(onehot_set2, min_support = 0.1, max_len = 2, use_colnames=True)
rules_set2 = association_rules(frequent_itemsets_set2)
rules_set2

CPU times: user 67.8 ms, sys: 4.36 ms, total: 72.2 ms
Wall time: 71 ms


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1000284),(1000112),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(1000112),(1000284),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(1000419),(1000112),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(1000112),(1000419),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(1001402),(1000112),1.0,1.0,1.0,1.0,1.0,0.0,inf
...,...,...,...,...,...,...,...,...,...
25117,(1142598),(1142592),1.0,1.0,1.0,1.0,1.0,0.0,inf
25118,(1142592),(1142859),1.0,1.0,1.0,1.0,1.0,0.0,inf
25119,(1142859),(1142592),1.0,1.0,1.0,1.0,1.0,0.0,inf
25120,(1142859),(1142598),1.0,1.0,1.0,1.0,1.0,0.0,inf


In [33]:
%%time
frequent_itemsets_set2 = apriori(onehot_set2, min_support = 0.1, max_len = 3, use_colnames=True)
rules_set2 = association_rules(frequent_itemsets_set2)
rules_set2

CPU times: user 18.9 s, sys: 962 ms, total: 19.9 s
Wall time: 19.9 s


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1000284),(1000112),1.0,1.0,1.0,1.0,1.0,0.0,inf
1,(1000112),(1000284),1.0,1.0,1.0,1.0,1.0,0.0,inf
2,(1000419),(1000112),1.0,1.0,1.0,1.0,1.0,0.0,inf
3,(1000112),(1000419),1.0,1.0,1.0,1.0,1.0,0.0,inf
4,(1001402),(1000112),1.0,1.0,1.0,1.0,1.0,0.0,inf
...,...,...,...,...,...,...,...,...,...
3969271,"(1142592, 1142598)",(1142859),1.0,1.0,1.0,1.0,1.0,0.0,inf
3969272,"(1142859, 1142598)",(1142592),1.0,1.0,1.0,1.0,1.0,0.0,inf
3969273,(1142592),"(1142859, 1142598)",1.0,1.0,1.0,1.0,1.0,0.0,inf
3969274,(1142859),"(1142592, 1142598)",1.0,1.0,1.0,1.0,1.0,0.0,inf
